# Coletar dados de API

Este notebook realiza a extração dos dados via API pública e salva os dados na camada Landing (LND), no formato JSON.

## Importações e configurações

In [8]:
import os
import zipfile
import json
import re
import shutil
import logging
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from typing import Any, List
import requests
from requests.exceptions import RequestException, HTTPError, Timeout, ConnectionError

## FUNÇÕES PARA EXTRAÇÃO DE DADOS

In [9]:
# Cria pasta de logs se ainda não existir
os.makedirs("../LND/logs", exist_ok=True)

# Caminho do arquivo de log
log_path = "../LND/logs/download.log"

# Configura o logging para salvar no caminho desejado
logging.basicConfig(
    filename=log_path,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"  # "w" para sobrescrever sempre
)

In [10]:
def download_and_extract_files(file_list: List[str], month: str = "2025-06", base_dir: str = "../LND") -> None:
    """
    Baixa e extrai arquivos .zip dos dados abertos da Receita Federal (CNPJ), organizando-os
    por tipo e mês em uma estrutura de diretórios padronizada. Os arquivos extraídos são 
    renomeados com timestamp e logs são gerados em cada etapa do processo.

    Args:
        file_list (List[str]): Lista com os nomes dos arquivos .zip a serem baixados.
        month (str, optional): Mês e ano dos dados no formato "YYYY-MM". Defaults to "2025-06".
        base_dir (str, optional): Caminho base onde os arquivos serão armazenados. Defaults to "../LND".

    Raises:
        HTTPError: Se a requisição do download falhar por erro HTTP.
        ConnectionError | Timeout: Em caso de falha de rede ou tempo de resposta excedido.
        RequestException: Para erros genéricos da biblioteca requests.
        zipfile.BadZipFile: Se o arquivo zip estiver corrompido.
        Exception: Para qualquer outro erro inesperado durante o processo.
    """
    base_url = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj"

    for zip_file_name in file_list:
        try:
            # Extrai o tipo do arquivo a partir do nome, ex: "Empresas", "Cnaes"
            match = re.match(r"([A-Za-z]+)", zip_file_name)
            file_type = match.group(1).lower() if match else "outros"

            # Monta a URL completa para o download do arquivo
            download_url = f"{base_url}/{month}/{zip_file_name}"

            # Cria timestamp para nomear arquivos extraídos com unicidade
            timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")

            # Define o diretório onde o arquivo será salvo, organizando por tipo e mês
            output_dir = os.path.join(base_dir, file_type, month)

            # Cria o diretório se não existir
            os.makedirs(output_dir, exist_ok=True)

            # Caminho completo do arquivo zip local para salvar/usar
            local_zip_path = os.path.join(output_dir, zip_file_name)

            # Verifica se o arquivo já foi baixado para evitar download repetido
            if os.path.exists(local_zip_path):
                msg = f"Arquivo já existe, pulando download: {zip_file_name}"
                tqdm.write(msg)
                logging.info(msg)
            else:
                # Informa início do download no console e no log
                tqdm.write(
                    f"Iniciando download de {zip_file_name} de {download_url}"
                )
                logging.info(
                    f"Iniciando download: {zip_file_name} de {download_url}"
                )

                # Requisição HTTP para download com streaming (para não carregar tudo na memória)
                response = requests.get(download_url, stream=True)

                # Levanta exceção para erros HTTP
                response.raise_for_status()

                # Obtém o tamanho do arquivo para barra de progresso
                file_size = int(response.headers.get("Content-Length", 0))

                # Abre arquivo local para escrita e exibe barra de progresso do tqdm
                with open(local_zip_path, "wb") as f, tqdm(
                    total=file_size,
                    unit='B',
                    unit_scale=True,
                    desc=f"Baixando {zip_file_name}",
                    ncols=80
                ) as pbar:
                    # Baixa em chunks e atualiza barra de progresso
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            pbar.update(len(chunk))

                # Log após download concluído com tamanho em MB
                logging.info(
                    f"Download concluído: {zip_file_name} ({file_size / (1024**2):.2f} MB)"
                )

            # Mensagem para extração do arquivo zip
            tqdm.write(f"Extraindo {zip_file_name}")
            logging.info(f"Iniciando extração: {zip_file_name}")

            # Abre o arquivo zip baixado para extração
            with zipfile.ZipFile(local_zip_path, "r") as zip_ref:

                # Itera sobre os arquivos dentro do zip
                for member in zip_ref.namelist():

                    # Ignora pastas dentro do zip (nomes que terminam com '/')
                    if not member.endswith("/"):

                        # Gera novo nome para o arquivo extraído com timestamp
                        base_name = os.path.splitext(zip_file_name)[0]
                        new_filename = f"{base_name}_{timestamp}.csv"
                        final_path = os.path.join(output_dir, new_filename)

                        # Copia o conteúdo extraído para o arquivo final (em blocos para evitar uso excessivo de memória)
                        with zip_ref.open(member) as source, open(final_path, "wb") as target:
                            shutil.copyfileobj(
                                source, target, length=1024 * 1024
                            )

                        # Informações de sucesso no console e log
                        tqdm.write(
                            f"Arquivo extraído e salvo como: {final_path}"
                        )
                        logging.info(f"Arquivo extraído e salvo: {final_path}")

            # Após extração bem-sucedida, remove o arquivo zip para liberar espaço
            os.remove(local_zip_path)
            tqdm.write(f"ZIP removido: {zip_file_name}")
            logging.info(f"ZIP removido: {zip_file_name}")

        # Tratamento de exceção para arquivos zip corrompidos
        except zipfile.BadZipFile as badzip:
            logging.error(
                f"Arquivo .zip corrompido: {zip_file_name} - {badzip}")

            # Remove o zip corrompido para tentar baixar novamente depois
            if os.path.exists(local_zip_path):
                os.remove(local_zip_path)
            continue  # Pula para o próximo arquivo

        # Tratamento para erros HTTP durante o download
        except HTTPError as http_err:
            logging.error(f"[HTTP ERROR] {response.status_code} - {http_err}")
            raise

        # Tratamento para erros de conexão ou timeout na rede
        except (ConnectionError, Timeout) as conn_err:
            logging.error(f"[ERRO DE CONEXÃO] - {conn_err}")
            raise

        # Tratamento para erros genéricos da biblioteca requests
        except RequestException as req_err:
            logging.error(f"[REQUEST ERROR] - {req_err}")
            raise

        # Captura e loga qualquer outro erro inesperado
        except Exception as e:
            logging.error(f"Erro inesperado ao processar {zip_file_name}: {e}")
            raise

In [11]:
def save_json(response: requests.Response, file_name: str, base_dir: str = "../LND") -> None:
    """
    Salva os dados de uma resposta JSON da API em um arquivo na estrutura de diretórios da camada LND.

    Args:
        response (requests.Response): Objeto de resposta da requisição HTTP.
        file_name (str): Nome da subpasta e prefixo do arquivo.
        base_dir (str): Diretório base onde os dados serão salvos (padrão: "../LND").

    Raises:
        HTTPError: Se a resposta contiver um status HTTP de erro.
        ValueError: Se o conteúdo não for um JSON válido.
        RequestException: Para outros erros relacionados a requests.
        Exception: Para outros erros de escrita/salvamento.

    Returns:
        None
    """
    try:
        # Verifica se a resposta HTTP foi bem-sucedida.
        # Se o status_code for >= 400, um HTTPError será lançado.
        response.raise_for_status()

        try:
            # Tenta converter o conteúdo da resposta para JSON.
            data: Any = response.json()
        except json.JSONDecodeError as e:
            # Se o conteúdo não for um JSON válido, lança um erro mais descritivo.
            raise ValueError("Resposta da API não é um JSON válido.") from e

        # Verifica se o JSON retornado é um dicionário ou uma lista.
        if not isinstance(data, (dict, list)):
            raise ValueError("Conteúdo JSON inválido: não é dict nem list.")

        # Gera timestamp para garantir nome único ao arquivo
        timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")

        # Cria o caminho onde o arquivo será salvo
        path = os.path.join(base_dir, file_name)

        # Cria o diretório, se ainda não existir
        os.makedirs(path, exist_ok=True)

        # Gera o nome completo do arquivo incluindo timestamp
        full_filename = os.path.join(path, f"{file_name}_{timestamp}.json")

        # Abre o arquivo no modo escrita e salva os dados JSON formatados
        with open(full_filename, "w", encoding="utf-8") as file:
            json.dump(data, file, indent=4, ensure_ascii=False)

        print(
            f"Arquivo JSON salvo com sucesso em: {os.path.abspath(full_filename)}")

    # Erro de status HTTP (ex: 404, 500, etc.)
    except HTTPError as http_err:
        print(f"[HTTP ERRO] Código {response.status_code}: {http_err}")
        raise

    # Falha de rede, servidor não encontrado, timeout, etc.
    except (ConnectionError, Timeout) as conn_err:
        print(f"[CONEXÃO ERRO] Falha na comunicação com a API: {conn_err}")
        raise

    # Erros genéricos da biblioteca requests
    except RequestException as req_err:
        print(f"[REQUEST ERRO] Erro inesperado com a requisição: {req_err}")
        raise

    # Qualquer outro erro inesperado
    except Exception as e:
        print(f"[X] Erro ao salvar JSON: {e}")
        raise

In [12]:
def list_files_rf(month: str = "2025-06") -> List[str]:
    """
    Lista os arquivos .zip disponíveis no diretório público da Receita Federal para o mês informado.

    A função acessa o diretório HTML dos dados abertos do CNPJ disponibilizados pela Receita Federal
    e extrai os nomes dos arquivos .zip disponíveis na pasta do mês especificado.

    Args:
        month (str, optional): Mês e ano no formato "YYYY-MM" que indica a pasta desejada. 
                               Padrão: "2025-06".

    Raises:
        HTTPError: Se a resposta HTTP indicar erro.
        RequestException: Para falhas gerais de conexão ou requisição.
        Exception: Para qualquer outro erro não previsto.

    Returns:
        List[str]: Lista contendo os nomes dos arquivos .zip encontrados.
    """
    try:
        # Define a URL base dos dados abertos do CNPJ
        base_url = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj"

        # Concatena a URL base com o mês informado para formar o link completo
        full_url = f"{base_url}/{month}/"

        # Faz requisição HTTP para acessar o conteúdo da página HTML da pasta do mês
        response = requests.get(full_url)

        # Lança erro automático se a resposta for código >= 400 (ex: 404, 500)
        response.raise_for_status()

        # Usa BeautifulSoup para interpretar o conteúdo HTML da resposta
        soup = BeautifulSoup(response.text, "html.parser")

        # Coleta todos os links <a href="..."> que terminam com ".zip"
        zip_files: List[str] = [
            a["href"]
            for a in soup.find_all("a", href=True)
            if a["href"].endswith(".zip")
        ]

        # Caso nenhum arquivo .zip seja encontrado, informa no terminal
        if not zip_files:
            print(
                f"[INFO] Nenhum arquivo .zip encontrado para o mês {month} em {full_url}."
            )

        # Retorna a lista de nomes de arquivos .zip encontrados
        return zip_files

    # Caso a resposta HTTP tenha código de erro (ex: 404), exibe mensagem
    except HTTPError as http_err:
        print(
            f"[HTTP ERRO] Falha ao acessar {full_url} - Código {response.status_code}: {http_err}")
        raise

    # Problemas de conexão ou tempo de espera excedido
    except (ConnectionError, Timeout) as conn_err:
        print(
            f"[CONEXÃO ERRO] Erro de conexão ao acessar {full_url}: {conn_err}")
        raise

    # Erros genéricos da biblioteca requests (DNS, SSL, etc.)
    except RequestException as req_err:
        print(
            f"[REQUEST ERRO] Erro inesperado na requisição ao acessar {full_url}: {req_err}")
        raise

    # Qualquer outro erro inesperado durante a execução
    except Exception as e:
        print(f"[X] Erro ao processar listagem de arquivos em {full_url}: {e}")
        raise

## Coleta dos dados

- API IBGE

In [6]:
try:
    url: str = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
    response: requests.Response = requests.get(url)
    save_json(response, "municipios")
except Exception as e:
    print(f"[FALHA TOTAL] {e}")

Arquivo JSON salvo com sucesso em: /home/wilcb/projeto_data_warehouse/LND/municipios/municipios_2025-07-23_223116.json


- Listar arquivos .zip do site da receita federal

In [13]:
files: List[str] = list_files_rf()
print("\n".join(files))

Cnaes.zip
Empresas0.zip
Empresas1.zip
Empresas2.zip
Empresas3.zip
Empresas4.zip
Empresas5.zip
Empresas6.zip
Empresas7.zip
Empresas8.zip
Empresas9.zip
Estabelecimentos0.zip
Estabelecimentos1.zip
Estabelecimentos2.zip
Estabelecimentos3.zip
Estabelecimentos4.zip
Estabelecimentos5.zip
Estabelecimentos6.zip
Estabelecimentos7.zip
Estabelecimentos8.zip
Estabelecimentos9.zip
Motivos.zip
Municipios.zip
Naturezas.zip
Paises.zip
Qualificacoes.zip
Simples.zip
Socios0.zip
Socios1.zip
Socios2.zip
Socios3.zip
Socios4.zip
Socios5.zip
Socios6.zip
Socios7.zip
Socios8.zip
Socios9.zip


In [ ]:
"""
    "Cnaes.zip",
    "Empresas1.zip",
    "Estabelecimentos1.zip",
    "Motivos.zip",
    "Municipios.zip",
    "Naturezas.zip",
    "Paises.zip",
    "Qualificacoes.zip",
    "Simples.zip",
    "Socios1.zip"
"""

files = [
    "Paises.zip",
    "Qualificacoes.zip",
    "Simples.zip",
    "Socios1.zip"
]

try:
    download_and_extract_files(files)
except Exception as e:
    print(f"[ERRO] Falha ao baixar e extrair arquivos: {e}")
    raise

Iniciando download de Paises.zip de https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2025-06/Paises.zip


Baixando Paises.zip: 100%|█████████████████| 2.75k/2.75k [00:00<00:00, 8.04MB/s]


Extraindo Paises.zip
Arquivo extraído e salvo como: ../LND/paises/2025-06/Paises_2025-07-23_212003.csv
ZIP removido: Paises.zip
Iniciando download de Qualificacoes.zip de https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2025-06/Qualificacoes.zip


Baixando Qualificacoes.zip: 100%|██████████████| 980/980 [00:00<00:00, 3.13MB/s]


Extraindo Qualificacoes.zip
Arquivo extraído e salvo como: ../LND/qualificacoes/2025-06/Qualificacoes_2025-07-23_212004.csv
ZIP removido: Qualificacoes.zip
Iniciando download de Simples.zip de https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2025-06/Simples.zip


Baixando Simples.zip: 100%|██████████████████| 261M/261M [02:33<00:00, 1.70MB/s]


Extraindo Simples.zip
Arquivo extraído e salvo como: ../LND/simples/2025-06/Simples_2025-07-23_212004.csv
ZIP removido: Simples.zip
Iniciando download de Socios1.zip de https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2025-06/Socios1.zip


Baixando Socios1.zip: 100%|████████████████| 49.5M/49.5M [00:29<00:00, 1.66MB/s]


Extraindo Socios1.zip
Arquivo extraído e salvo como: ../LND/socios/2025-06/Socios1_2025-07-23_212245.csv
ZIP removido: Socios1.zip


## Dados IBGE

Os dados complementares necessários para este projeto — como Natureza Jurídica, CNAE, Simples Nacional, Municípios da Receita, entre outros — não estão disponíveis via API pública. No entanto, podem ser obtidos diretamente no portal da Receita Federal através de arquivos CSV, ou é só fazer o download atráves da função acima.

📂 Acesse: [Base CNPJ — Receita Federal (jun/2025)](https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2025-06/)

Esses arquivos devem ser considerados parte da **camada Landing (LND)** e posteriormente processados nos notebooks seguintes.